# Baseline Model

This is a initial design of our background music discriminator model. Which would just have 2 fc layers for source subnets and 1 simple LSTM RNN.

In [1]:
from ipynb.fs.full.DataProcessUtils import readDataSet
import tensorflow as tf
import numpy as np

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [16]:
data = readDataSet("../data/5s_clips", 48000, 20)

----- executing ------
001253.mp4
----- succeeded -------
----- executing ------
004693.mp4
----- succeeded -------
----- executing ------
003265.mp4
----- invalid clip, deleted -------
----- executing ------
005155.mp4
----- succeeded -------
----- executing ------
002493.mp4
----- succeeded -------
----- executing ------
001683.mp4
----- succeeded -------
----- executing ------
000228.mp4
----- succeeded -------
----- executing ------
006650.mp4
----- succeeded -------
----- executing ------
005731.mp4
----- succeeded -------
----- executing ------
005704.mp4
----- succeeded -------
----- executing ------
004655.mp4
----- succeeded -------
----- executing ------
000502.mp4
----- succeeded -------
----- executing ------
003609.mp4
----- succeeded -------
----- executing ------
003201.mp4
----- succeeded -------
----- executing ------
000439.mp4
----- succeeded -------
----- executing ------
001634.mp4
----- succeeded -------
----- executing ------
001574.mp4
----- succeeded -------
--

## Utils

In [2]:
def get_variable(name, shape, alpha=0.01):
    initial = alpha * tf.random_normal(shape)
    return tf.Variable(initial, name=name)

In [30]:
def get_optimizer(labels, logits, learning_rate = 10**-7):
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name='Adam').minimize(loss)
    return loss, optimizer

In [4]:
def get_accuracy(labels, logits):
    prediction = tf.cast(tf.greater(logits, 0.5), dtype=tf.float32)
    correct = tf.equal(labels, prediction)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    return accuracy

## Video Subnet

In [5]:
def video_subnet(x, w, b):
    P0 = tf.reshape(x, [-1, 1])
    Z1 = tf.add(tf.matmul(w, P0), b)
    A1 = tf.nn.relu(Z1)
    
    return A1

In [6]:
def video_fn(w, b):
    def video_fn_1(x):
        return tf.reshape(video_subnet(x, w, b), [-1])
    return video_fn_1

In [7]:
def video_model(w, b):
    def video_model_1(x):
        return tf.map_fn(video_fn(w, b), x)
    return video_model_1

## Audio Subnet

In [8]:
def audio_subnet(x, w, b):
    P0 = tf.reshape(x, [-1, 1])
    Z1 = tf.add(tf.matmul(w, P0), b)
    A1 = tf.nn.relu(Z1)
    
    return A1

In [9]:
def audio_fn(w, b):
    def audio_fn_1(x):
        return tf.reshape(audio_subnet(x, w, b), [-1])
    return audio_fn_1

In [10]:
def audio_model(w, b):
    def audio_model_1(x):
        return tf.map_fn(audio_fn(w, b), x)
    return audio_model_1

## Rnn

In [11]:
def rnn_model(x, w, b, rnn_size, width, height):
    # LSTM
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(rnn_size, reuse=False)
    
    # D0 is the number of pictures, here we want to split the pictures into sequences of lines
    # this transpose would make row number the 1st dimension
    x = tf.transpose(x, [1, 0, 2])
    
    # reshape to make the tensor in shape [ROW * #_OF_PICS, COL]
    x = tf.reshape(x, [-1, width])
    
    # split the tensor into ROW groups, each group has shape [#_OF_PICS, COL]
    # so group 1 contains row 1 from all pics, group 2 contains row 2 from all pics
    x = tf.split(x, height)
    
    # get the outputs, which would be the last output from LSTM
    outputs, status = tf.nn.static_rnn(lstm_cell, x, dtype=tf.float32, scope=tf.get_variable_scope())
    logits = tf.add(tf.matmul(outputs[-1], w), b)

    return logits

## Main Model

In [27]:
def cs230_model(frames, v, v_height, v_width, v_features, a, a_height, a_width, a_features, rnn_size):
    
    vw = get_variable('video_w', [v_features, v_height * v_width * 3])
    vb = get_variable('video_b', [v_features, 1])
    aw = get_variable('audio_w', [a_features, a_height * a_width])
    ab = get_variable('audio_b', [a_features, 1])
    rw = get_variable('rnn_w', [rnn_size, 1])
    rb = get_variable('rnn_b', [1])
    
    v_out = tf.map_fn(video_model(vw, vb), v, name='video_features') # --> [None, frames, v_features]
    a_out = tf.map_fn(audio_model(aw, ab), a, name='audio_features') # --> [None, frames, a_features]
    
    c = tf.concat([v_out, a_out], axis=2, name='concat_features')     # --> [None, frames, v_features + a_features]
    tf.summary.tensor_summary('concat', c)
    
    logits = rnn_model(c, rw, rb, rnn_size, v_features + a_features, frames)
    
    return logits

In [29]:
def test_train(iteration):
    data = readDataSet("../data/5s_clips", 48000, 2)
    
    videos = np.asarray([d[1] for d in data])
    audios = np.asarray([d[2] for d in data])
    
    frames, v_height, v_width, dummy_0 = videos[0].shape
    dummy_0, a_height, a_width = audios[0].shape
    
    y_truth = np.asarray([[0], [1]])
    y = tf.placeholder(tf.float32, [None, 1])
    v = tf.placeholder(tf.float32, [None, frames, v_height, v_width, 3])
    a = tf.placeholder(tf.float32, [None, frames, a_height, a_width])
    y_hat = cs230_model(frames, v, v_height, v_width, 32, a, a_height, a_width, 16, 32)
    
    los, opt = get_optimizer(y, y_hat)
    acc = get_accuracy(y, y_hat)
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    # summary part
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter('../models/baseline_model/train', sess.graph)
    
    for i in range(iteration):
        summary, loss, dummy_0, accuracy = sess.run([merged, los, opt, acc], feed_dict={v: videos, a: audios, y:y_truth})
        print(str(i) + " loss: " + str(loss) + " accuracy: " + str(accuracy))
        train_writer.add_summary(summary, i)

In [14]:
# test_train(10)

In [31]:
def test_train_alignment(iteration, data):
    train_size = 10
    dev_size = 10
    train_half = train_size // 2
    dev_half = dev_size // 2
    total = train_size + dev_size
    
    videos = [d[1] for d in data]
    audios = [d[2][:, :, :] for d in data]
    
    v_train_clips = videos[:train_size]
    a_train_clips = audios[:train_size]
    v_dev_clips = videos[train_size:total]
    a_dev_clips = audios[train_size:total]
    
    for i in range(train_half):
        # cut the first 96 frames
        v_train_clips[i] = np.asarray(v_train_clips[i][:96, :, :, :])
        a_train_clips[i] = np.asarray(a_train_clips[i][:96, :, :])
        # shift the audio of second half of trainning data by half second
        v_train_clips[i + train_half] = np.asarray(v_train_clips[i + train_half][:96, :, :, :])
        a_train_clips[i + train_half] = np.asarray(a_train_clips[i + train_half][12:108, :, :])
        
    # for i in range(train_size):
    #     print(v_train_clips[i].shape)
    #     print(a_train_clips[i].shape)
    
    y_train = np.reshape(np.asarray([1 - (i // train_half) for i in range(train_size)], dtype=np.float), (train_size, 1))
    v_train = np.asarray(v_train_clips, dtype=np.float) + 0
    a_train = np.asarray(a_train_clips, dtype=np.float) + 0
    
    # for i in range(2):
    #     for j in range(96):
    #         for k in range(129):
    #             print(str(i) + "." + str(j) + "." + str(k) + ":" + str(a_train[i][j][k]))
    
    for i in range(dev_half):
        # cut the first 96 frames
        v_dev_clips[i] = v_dev_clips[i][:96, :, :, :]
        a_dev_clips[i] = a_dev_clips[i][:96, :, :]
        # shift the audio of second half of trainning data by half second
        v_dev_clips[i + dev_half] = v_dev_clips[i + dev_half][:96, :, :, :]
        a_dev_clips[i + dev_half] = a_dev_clips[i + dev_half][12:108, :, :]
    
    # for i in range(dev_size):
    #     print(v_dev_clips[i].shape)
    #     print(a_dev_clips[i].shape)
    
    y_dev = np.reshape(np.asarray([1 - (i // dev_half) for i in range(dev_size)], dtype=np.float), (dev_size, 1))
    v_dev = np.array(v_dev_clips, dtype=np.float)
    a_dev = np.array(a_dev_clips, dtype=np.float)
    
    frames, v_height, v_width, dummy_0 = v_train[0].shape
    dummy_0, a_height, a_width = a_train[0].shape
    
    yy = tf.placeholder(tf.float32, [None, 1], name="y")
    vv = tf.placeholder(tf.float32, [None, frames, v_height, v_width, 3], name="videos")
    aa = tf.placeholder(tf.float32, [None, frames, a_height, a_width], name="audios")
    y_hat = cs230_model(frames, vv, v_height, v_width, 64, aa, a_height, a_width, 16, 64)
    
    los, opt = get_optimizer(yy, y_hat, 2*10**-6)
    acc = get_accuracy(yy, y_hat)
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    # summary part
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter('../models/baseline_model_with_alignment/train', sess.graph)
    
    tf.reset_default_graph()
    try:
        for i in range(iteration):
            summary, loss, dummy_0 = sess.run([merged, los, opt], feed_dict={vv:v_train, aa:a_train, yy:y_train})
            print(str(i) + " loss: " + str(loss))
            if i % 20 == 19:
                # run the dev set every 20 iterations
                accuracy = sess.run([acc], feed_dict={vv:v_dev, aa:a_dev, yy:y_dev})
                print(str(i) + " acc: " + str(accuracy))
            train_writer.add_summary(summary, i)
    finally:
        sess.close()

In [32]:
test_train_alignment(10000, data)

0 loss: 0.6936151
1 loss: 0.6899953
2 loss: 0.68806845
3 loss: 0.6868587
4 loss: 0.6846045
5 loss: 0.68423843
6 loss: 0.6839389
7 loss: 0.68281627
8 loss: 0.68227184
9 loss: 0.68184435
10 loss: 0.6814694
11 loss: 0.681371
12 loss: 0.68072903
13 loss: 0.6801936
14 loss: 0.67990303
15 loss: 0.68024766
16 loss: 0.67940223
17 loss: 0.6792203
18 loss: 0.6790975
19 loss: 0.6790905
19 acc: [0.5]
20 loss: 0.67809796
21 loss: 0.678058
22 loss: 0.67807347
23 loss: 0.67803246
24 loss: 0.6779127
25 loss: 0.6778801
26 loss: 0.6777067
27 loss: 0.67767966
28 loss: 0.6776917
29 loss: 0.6777608
30 loss: 0.6777573
31 loss: 0.677477
32 loss: 0.6770223
33 loss: 0.67698765
34 loss: 0.677024
35 loss: 0.67697215
36 loss: 0.67688924
37 loss: 0.6768196
38 loss: 0.676756
39 loss: 0.67668855
39 acc: [0.5]
40 loss: 0.6766105
41 loss: 0.6764409
42 loss: 0.6761851
43 loss: 0.67605984
44 loss: 0.6759318
45 loss: 0.6758459
46 loss: 0.6757928
47 loss: 0.6757514
48 loss: 0.67571676
49 loss: 0.67568374
50 loss: 0.675643

395 loss: 0.66138715
396 loss: 0.66135675
397 loss: 0.66131985
398 loss: 0.66129225
399 loss: 0.66127336
399 acc: [0.5]
400 loss: 0.6612582
401 loss: 0.661245
402 loss: 0.6612324
403 loss: 0.6612196
404 loss: 0.66120625
405 loss: 0.66119176
406 loss: 0.66117555
407 loss: 0.6611558
408 loss: 0.66112965
409 loss: 0.6610905
410 loss: 0.6610281
411 loss: 0.6609332
412 loss: 0.6607837
413 loss: 0.6606499
414 loss: 0.66063404
415 loss: 0.6606482
416 loss: 0.6606475
417 loss: 0.66062486
418 loss: 0.66059566
419 loss: 0.6605702
419 acc: [0.5]
420 loss: 0.660551
421 loss: 0.66053617
422 loss: 0.6605228
423 loss: 0.66050977
424 loss: 0.66049635
425 loss: 0.6604823
426 loss: 0.6604684
427 loss: 0.66045487
428 loss: 0.6604419
429 loss: 0.66042936
430 loss: 0.6604172
431 loss: 0.6604053
432 loss: 0.66039354
433 loss: 0.6603819
434 loss: 0.66037047
435 loss: 0.66035926
436 loss: 0.6603481
437 loss: 0.66033715
438 loss: 0.66032636
439 loss: 0.6603156
439 acc: [0.5]
440 loss: 0.6603049
441 loss: 0.660

783 loss: 0.6537524
784 loss: 0.6537458
785 loss: 0.65375054
786 loss: 0.6537473
787 loss: 0.65374154
788 loss: 0.65372425
789 loss: 0.6536989
790 loss: 0.6536591
791 loss: 0.65356076
792 loss: 0.65347815
793 loss: 0.65345633
794 loss: 0.65344155
795 loss: 0.65342253
796 loss: 0.6534065
797 loss: 0.6534618
798 loss: 0.6551137
799 loss: 0.6563464
799 acc: [0.5]
800 loss: 0.6572784
801 loss: 0.6583994
802 loss: 0.6570568
803 loss: 0.6572259
804 loss: 0.6563923
805 loss: 0.6564522
806 loss: 0.6552213
807 loss: 0.6548077
808 loss: 0.655358
809 loss: 0.6557709
810 loss: 0.65514237
811 loss: 0.65489036
812 loss: 0.6568147
813 loss: 0.6589426
814 loss: 0.65929466
815 loss: 0.6582684
816 loss: 0.6573345
817 loss: 0.6551606
818 loss: 0.6547898
819 loss: 0.65476644
819 acc: [0.5]
820 loss: 0.65356696
821 loss: 0.6525103
822 loss: 0.6512402
823 loss: 0.65083814
824 loss: 0.65078604
825 loss: 0.6503333
826 loss: 0.650077
827 loss: 0.6498195
828 loss: 0.64964926
829 loss: 0.6496881
830 loss: 0.6493

1164 loss: 0.6402258
1165 loss: 0.64021367
1166 loss: 0.64020145
1167 loss: 0.6401893
1168 loss: 0.64017713
1169 loss: 0.640165
1170 loss: 0.6401528
1171 loss: 0.64014065
1172 loss: 0.6401285
1173 loss: 0.6401164
1174 loss: 0.64010423
1175 loss: 0.6400921
1176 loss: 0.64008
1177 loss: 0.64006776
1178 loss: 0.6400557
1179 loss: 0.6400436
1179 acc: [0.5]
1180 loss: 0.64003146
1181 loss: 0.6400193
1182 loss: 0.6400072
1183 loss: 0.6399951
1184 loss: 0.639983
1185 loss: 0.6399709
1186 loss: 0.63995886
1187 loss: 0.63994676
1188 loss: 0.63993466
1189 loss: 0.63992274
1190 loss: 0.6399106
1191 loss: 0.6398984
1192 loss: 0.6398864
1193 loss: 0.63987434
1194 loss: 0.63986236
1195 loss: 0.6398502
1196 loss: 0.6398381
1197 loss: 0.63982606
1198 loss: 0.6398141
1199 loss: 0.6398019
1199 acc: [0.5]
1200 loss: 0.63978994
1201 loss: 0.63977784
1202 loss: 0.63976574
1203 loss: 0.6397537
1204 loss: 0.6397417
1205 loss: 0.6397296
1206 loss: 0.6397176
1207 loss: 0.63970554
1208 loss: 0.6396935
1209 loss

1535 loss: 0.6332998
1536 loss: 0.63328695
1537 loss: 0.6332742
1538 loss: 0.6332613
1539 loss: 0.6332482
1539 acc: [0.5]
1540 loss: 0.6332349
1541 loss: 0.6332218
1542 loss: 0.6332087
1543 loss: 0.6331955
1544 loss: 0.63318235
1545 loss: 0.6331691
1546 loss: 0.633156
1547 loss: 0.633143
1548 loss: 0.6331299
1549 loss: 0.6331169
1550 loss: 0.63310397
1551 loss: 0.6330909
1552 loss: 0.6330779
1553 loss: 0.63306504
1554 loss: 0.63305205
1555 loss: 0.6330392
1556 loss: 0.6330263
1557 loss: 0.63301337
1558 loss: 0.6330005
1559 loss: 0.6329876
1559 acc: [0.5]
1560 loss: 0.63297474
1561 loss: 0.63296187
1562 loss: 0.632949
1563 loss: 0.6329361
1564 loss: 0.6329233
1565 loss: 0.63291043
1566 loss: 0.6328977
1567 loss: 0.63288486
1568 loss: 0.632872
1569 loss: 0.63285923
1570 loss: 0.6328464
1571 loss: 0.6328336
1572 loss: 0.63282084
1573 loss: 0.632808
1574 loss: 0.6327953
1575 loss: 0.63278246
1576 loss: 0.63276976
1577 loss: 0.632757
1578 loss: 0.6327442
1579 loss: 0.63273144
1579 acc: [0.5

...

8239 acc: [0.5]
8240 loss: 0.5246376
8241 loss: 0.5246218
8242 loss: 0.5246061
8243 loss: 0.5245904
8244 loss: 0.5245744
8245 loss: 0.5245579
8246 loss: 0.5245403
8247 loss: 0.5245202
8248 loss: 0.5244938
8249 loss: 0.5244466
8250 loss: 0.52429473
8251 loss: 0.5236763
8252 loss: 0.52375704
8253 loss: 0.5234615
8254 loss: 0.5237037
8255 loss: 0.52381665
8256 loss: 0.5238202
8257 loss: 0.5236925
8258 loss: 0.5235008
8259 loss: 0.52344716
8259 acc: [0.5]
8260 loss: 0.5234291
8261 loss: 0.52337766
8262 loss: 0.5232905
8263 loss: 0.5232296
8264 loss: 0.5232011
8265 loss: 0.52317894
8266 loss: 0.52316135
8267 loss: 0.52314913
8268 loss: 0.52313185
8269 loss: 0.5231103
8270 loss: 0.5230896
8271 loss: 0.52306974
8272 loss: 0.52305096
8273 loss: 0.5230331
8274 loss: 0.52301604
8275 loss: 0.52299947
8276 loss: 0.5229834
8277 loss: 0.5229675
8278 loss: 0.5229521
8279 loss: 0.5229369
8279 acc: [0.5]
8280 loss: 0.5229219
8281 loss: 0.52290714
8282 loss: 0.52289265
8283 loss: 0.5228783
8284 loss: 0.

8610 loss: 0.517393
8611 loss: 0.5173795
8612 loss: 0.51736605
8613 loss: 0.5173526
8614 loss: 0.5173391
8615 loss: 0.5173257
8616 loss: 0.5173122
8617 loss: 0.5172988
8618 loss: 0.5172852
8619 loss: 0.5172718
8619 acc: [0.5]
8620 loss: 0.51725835
8621 loss: 0.51724494
8622 loss: 0.51723146
8623 loss: 0.517218
8624 loss: 0.5172046
8625 loss: 0.51719123
8626 loss: 0.51717776
8627 loss: 0.51716435
8628 loss: 0.5171509
8629 loss: 0.5171374
8630 loss: 0.517124
8631 loss: 0.5171105
8632 loss: 0.5170971
8633 loss: 0.51708376
8634 loss: 0.5170703
8635 loss: 0.5170569
8636 loss: 0.5170435
8637 loss: 0.51703
8638 loss: 0.51701653
8639 loss: 0.5170032
8639 acc: [0.5]
8640 loss: 0.51698977
8641 loss: 0.51697636
8642 loss: 0.5169629
8643 loss: 0.5169495
8644 loss: 0.5169361
8645 loss: 0.5169227
8646 loss: 0.51690924
8647 loss: 0.5168959
8648 loss: 0.5168824
8649 loss: 0.51686907
8650 loss: 0.5168556
8651 loss: 0.5168422
8652 loss: 0.5168289
8653 loss: 0.5168154
8654 loss: 0.516802
8655 loss: 0.516

8981 loss: 0.5124848
8982 loss: 0.5124718
8983 loss: 0.5124587
8984 loss: 0.5124457
8985 loss: 0.5124327
8986 loss: 0.51241964
8987 loss: 0.51240665
8988 loss: 0.5123936
8989 loss: 0.51238054
8990 loss: 0.51236755
8991 loss: 0.5123545
8992 loss: 0.5123415
8993 loss: 0.51232845
8994 loss: 0.51231545
8995 loss: 0.51230246
8996 loss: 0.5122894
8997 loss: 0.51227635
8998 loss: 0.5122633
8999 loss: 0.5122503
8999 acc: [0.5]
9000 loss: 0.51223737
9001 loss: 0.5122243
9002 loss: 0.5122113
9003 loss: 0.5121982
9004 loss: 0.5121852
9005 loss: 0.5121722
9006 loss: 0.5121592
9007 loss: 0.5121462
9008 loss: 0.51213324
9009 loss: 0.5121202
9010 loss: 0.5121072
9011 loss: 0.5120942
9012 loss: 0.5120812
9013 loss: 0.51206815
9014 loss: 0.51205516
9015 loss: 0.51204216
9016 loss: 0.5120292
9017 loss: 0.5120162
9018 loss: 0.5120031
9019 loss: 0.5119902
9019 acc: [0.5]
9020 loss: 0.5119772
9021 loss: 0.51196414
9022 loss: 0.51195115
9023 loss: 0.51193815
9024 loss: 0.51192516
9025 loss: 0.5119122
9026 l

9352 loss: 0.50631607
9353 loss: 0.506303
9354 loss: 0.5062901
9355 loss: 0.5062771
9356 loss: 0.50626403
9357 loss: 0.50625104
9358 loss: 0.5062381
9359 loss: 0.5062251
9359 acc: [0.5]
9360 loss: 0.50621206
9361 loss: 0.5061991
9362 loss: 0.5061861
9363 loss: 0.5061732
9364 loss: 0.50616014
9365 loss: 0.5061472
9366 loss: 0.50613415
9367 loss: 0.5061212
9368 loss: 0.5061082
9369 loss: 0.50609523
9370 loss: 0.50608224
9371 loss: 0.5060693
9372 loss: 0.5060563
9373 loss: 0.5060433
9374 loss: 0.5060304
9375 loss: 0.5060174
9376 loss: 0.50600445
9377 loss: 0.50599146
9378 loss: 0.50597847
9379 loss: 0.50596553
9379 acc: [0.5]
9380 loss: 0.50595254
9381 loss: 0.50593954
9382 loss: 0.5059266
9383 loss: 0.5059136
9384 loss: 0.5059007
9385 loss: 0.5058877
9386 loss: 0.50587475
9387 loss: 0.50586176
9388 loss: 0.50584877
9389 loss: 0.50583583
9390 loss: 0.50582284
9391 loss: 0.5058099
9392 loss: 0.50579697
9393 loss: 0.5057839
9394 loss: 0.50577104
9395 loss: 0.5057581
9396 loss: 0.5057451
939

9721 loss: 0.5015734
9722 loss: 0.5015606
9723 loss: 0.50154793
9724 loss: 0.50153524
9725 loss: 0.5015224
9726 loss: 0.5015098
9727 loss: 0.501497
9728 loss: 0.5014843
9729 loss: 0.5014715
9730 loss: 0.5014588
9731 loss: 0.5014461
9732 loss: 0.50143343
9733 loss: 0.5014206
9734 loss: 0.501408
9735 loss: 0.5013952
9736 loss: 0.50138247
9737 loss: 0.5013698
9738 loss: 0.5013571
9739 loss: 0.5013444
9739 acc: [0.5]
9740 loss: 0.5013316
9741 loss: 0.50131893
9742 loss: 0.5013062
9743 loss: 0.50129354
9744 loss: 0.5012808
9745 loss: 0.5012681
9746 loss: 0.50125533
9747 loss: 0.50124264
9748 loss: 0.50122994
9749 loss: 0.50121725
9750 loss: 0.5012045
9751 loss: 0.5011918
9752 loss: 0.5011791
9753 loss: 0.5011664
9754 loss: 0.50115365
9755 loss: 0.50114095
9756 loss: 0.5011283
9757 loss: 0.5011155
9758 loss: 0.50110286
9759 loss: 0.50109017
9759 acc: [0.5]
9760 loss: 0.5010774
9761 loss: 0.5010648
9762 loss: 0.501052
9763 loss: 0.5010393
9764 loss: 0.50102663
9765 loss: 0.50101393
9766 loss:

In [ ]:
for i in range(500):
    print(data[i][1].shape)

In [ ]:
a = tf.constant(
    [
        [
            [
                [[1, 2], [3, 4]], [[5, 6], [7, 8]]
            ], 
            [
                [[9, 10], [11, 12]], [[13, 14], [15, 16]]
            ], 
            [
                [[17, 18], [19, 20]], [[21, 22], [23, 24]]
            ]
        ],
        [
            [
                [[1, 2], [3, 4]], [[5, 6], [7, 8]]
            ], 
            [
                [[9, 10], [11, 12]], [[13, 14], [15, 16]]
            ], 
            [
                [[17, 18], [19, 20]], [[21, 22], [23, 24]]
            ]
        ]
    ]
)
b = tf.constant(
    [
        [
            [
                [1, 2], [3, 4]
            ], 
            [
                [9, 10], [11, 12]
            ], 
            [
                [17, 18], [19, 20]
            ]
        ],
        [
            [
                [1, 2], [3, 4]
            ], 
            [
                [9, 10], [11, 12]
            ], 
            [
                [17, 18], [19, 20]
            ]
        ]
    ]
)

In [ ]:
d = tf.reshape(a, [-1])

In [ ]:
ww1 = tf.constant([[1, 2, 1, 2, 1, 2, 1, 2], [1, 2, 1, 2, 1, 2, 1, 2]])
ww2 = tf.constant([[1, 2, 3, 4]])
bb1 = tf.constant([[1], [2]])
bb2 = tf.constant([1])
oo1 = tf.map_fn(video_model(ww1, bb1), a)
oo2 = tf.map_fn(video_model(ww2, bb2), b)
cc = tf.concat([oo1, oo2], axis=2)
ss = tf.shape(cc)
sa = tf.shape(oo1)
sb = tf.shape(oo2)

In [ ]:
sess = tf.Session()

In [ ]:
sess.run(cc)

In [ ]:
sess.run(cc)